# Pandas is powerful, how to use it?
A convenient tool and usage demo to do data cleaning using Pandas DataFrame

In [10]:
from __future__ import annotations
import numpy as np
import pandas as pd
from typing import Dict, Any, List
from datetime import datetime


# Transform a pandas df while keeping track of the history of transformations to allow reverting back to earlier state.

In [30]:
class DataCleaner:
  def __init__(self, df: pd.DataFrame):
    self.df = df
    self.current = self.df.copy()
    self.history =[]

  # a helper method to convert datatime format
  def to_datetime64(dt): # dt in the format of timestamp
    return np.datetime64(datetime.strptime(str(dt),'%Y-%m-%d %H:%M:%S'))
  
  # adjust the data type of columns 
  def adjust_dtype(self, types: Dict[str, Any]) -> None:
    for type1 in types: # assuming the input could be for more than one column
      if type1 == "timestamp":     
        self.df['timestamp'] = self.df['timestamp'].apply(to_datetime64)
        return
    return
  
  # imputate with missing values
  def impute_missing(self, columns: List[str]) -> None:
    for c in columns:
      if c == 'amount':
        self.df.loc[self.df.loc[:,c].isnull(),c]=self.df.loc[:,c].mean()
        self.null_amount_idx =  self.df.index[self.df['amount'].isnull()].tolist()
  
  # revert df back to an earlier state 
  def revert(self, steps_back: int = 1) -> None:
    length = len(self.history)
    txt, df1 = self.history[length - steps_back -1]
    self.current = df1.copy()
    self.history = self.history[0:(length -1 - steps_back)]

   # save the class object using pickle
  def save(self, path: str) -> None:
    self.df.to_pickle(path)

    #
  def load(path: str) -> DataCleaner:
    return DataCleaner(pd.read_pickle(path))    

In [31]:
transactions = pd.DataFrame(
    {
        "customer_id": [10, 10, 13, 10, 11, 11, 10],
        "amount": [1.00, 1.31, 20.5, 0.5, 0.2, 0.2, np.nan], #  "amount": [1.00, 1.31, 20.5, 0.5, 0.2, np.nan, np.nan],
        "timestamp": [
            "2020-10-08 11:32:01",
            "2020-10-08 13:45:00",
            "2020-10-07 05:10:30",
            "2020-10-08 12:30:00",
            "2020-10-07 01:29:33",
            "2020-10-08 13:45:00",
            "2020-10-09 02:05:21",
        ]
    }
)

In [13]:
print(transactions.columns)
print(transactions.index )

Index(['customer_id', 'amount', 'timestamp'], dtype='object')
RangeIndex(start=0, stop=7, step=1)


In [32]:
transactions_dc = DataCleaner(transactions)
transactions_dc.current = transactions_dc.df.copy()
transactions_dc.history.append(('Initial df',transactions_dc.current) )
print(f"Current dataframe:\n{transactions_dc.current}")

Current dataframe:
   customer_id  amount            timestamp
0           10    1.00  2020-10-08 11:32:01
1           10    1.31  2020-10-08 13:45:00
2           13   20.50  2020-10-07 05:10:30
3           10    0.50  2020-10-08 12:30:00
4           11    0.20  2020-10-07 01:29:33
5           11    0.20  2020-10-08 13:45:00
6           10     NaN  2020-10-09 02:05:21


In [28]:
print(f"Initial dtypes:\n{transactions_dc.current.dtypes}")

Initial dtypes:
customer_id      int64
amount         float64
timestamp       object
dtype: object


In [33]:
transactions_dc.adjust_dtype({"timestamp": np.datetime64})
transactions_dc.current = transactions_dc.df.copy()
transactions_dc.history.append(("Adjusted dtypes using {'timestamp': <class 'numpy.datetime64'>}",transactions_dc.current) )

print(f"Changed dtypes to:\n{transactions_dc.current.dtypes}")
#print(transactions_dc.history)

Changed dtypes to:
customer_id             int64
amount                float64
timestamp      datetime64[ns]
dtype: object


In [34]:
transactions_dc.impute_missing(columns=["amount"])
transactions_dc.current = transactions_dc.df.copy()
transactions_dc.history.append(("Imputed missing in ['amount']",transactions_dc.current) )
print(f"Imputed missing as overall mean:\n{transactions_dc.current}")

Imputed missing as overall mean:
   customer_id     amount           timestamp
0           10   1.000000 2020-10-08 11:32:01
1           10   1.310000 2020-10-08 13:45:00
2           13  20.500000 2020-10-07 05:10:30
3           10   0.500000 2020-10-08 12:30:00
4           11   0.200000 2020-10-07 01:29:33
5           11   0.200000 2020-10-08 13:45:00
6           10   3.951667 2020-10-09 02:05:21


In [35]:
print(f"History of changes:\n{transactions_dc.history}")

History of changes:
[('Initial df',    customer_id  amount            timestamp
0           10    1.00  2020-10-08 11:32:01
1           10    1.31  2020-10-08 13:45:00
2           13   20.50  2020-10-07 05:10:30
3           10    0.50  2020-10-08 12:30:00
4           11    0.20  2020-10-07 01:29:33
5           11    0.20  2020-10-08 13:45:00
6           10     NaN  2020-10-09 02:05:21), ("Adjusted dtypes using {'timestamp': <class 'numpy.datetime64'>}",    customer_id  amount           timestamp
0           10    1.00 2020-10-08 11:32:01
1           10    1.31 2020-10-08 13:45:00
2           13   20.50 2020-10-07 05:10:30
3           10    0.50 2020-10-08 12:30:00
4           11    0.20 2020-10-07 01:29:33
5           11    0.20 2020-10-08 13:45:00
6           10     NaN 2020-10-09 02:05:21), ("Imputed missing in ['amount']",    customer_id     amount           timestamp
0           10   1.000000 2020-10-08 11:32:01
1           10   1.310000 2020-10-08 13:45:00
2           13  20.50000

In [36]:
transactions_dc.save("transactions")
loaded_dc = DataCleaner.load("transactions")
print(f"Loaded DataCleaner current df:\n{loaded_dc.current}")

Loaded DataCleaner current df:
   customer_id     amount           timestamp
0           10   1.000000 2020-10-08 11:32:01
1           10   1.310000 2020-10-08 13:45:00
2           13  20.500000 2020-10-07 05:10:30
3           10   0.500000 2020-10-08 12:30:00
4           11   0.200000 2020-10-07 01:29:33
5           11   0.200000 2020-10-08 13:45:00
6           10   3.951667 2020-10-09 02:05:21


In [37]:
transactions_dc.revert()
print(f"Reverting missing value imputation:\n{transactions_dc.current}")

Reverting missing value imputation:
   customer_id  amount           timestamp
0           10    1.00 2020-10-08 11:32:01
1           10    1.31 2020-10-08 13:45:00
2           13   20.50 2020-10-07 05:10:30
3           10    0.50 2020-10-08 12:30:00
4           11    0.20 2020-10-07 01:29:33
5           11    0.20 2020-10-08 13:45:00
6           10     NaN 2020-10-09 02:05:21
